O modelo está praticamente respondendo o que existe na coluna connection_status. Podemos separar em duas análises:
- Quais fatores contam quando está conectado
- Quais fatores contam quando não está conectado

In [ ]:
categorical = [
    'regional', 
    'client_class',
    'disconnection_executed'
]
numerical = [
    'total_debt',
    'total_bills'
]
datetime = [
    'disconnection_date'
]
df_n_connected = df.loc[df.connection_status == 'connected'].reset_index(drop=True).drop(['connection_status'], axis = 1)
df_n_disconnected = df.loc[df.connection_status == 'disconnected'].reset_index(drop=True).drop(['connection_status'], axis = 1)

p_connected = Pipeline(df_n_connected, categorical, numerical, 'collected')
p_connected.preprocessing()

p_disconnected = Pipeline(df_n_disconnected, categorical, numerical, 'collected')
p_disconnected.preprocessing()

In [ ]:
print(f'''O df composto pelos conectados possui shape {df_n_connected.shape}
O número de positivos para esse df é de {df_n_connected.collected.sum():.0f}, representando {df_n_connected.collected.sum()/df_n_connected.shape[0]*100:.2f}%
------
O df composto pelos desconectados possui shape {df_n_disconnected.shape}
O número de positivos para esse df é de {df_n_disconnected.collected.sum():.0f}, representando {df_n_disconnected.collected.sum()/df_n_disconnected.shape[0]*100:.2f}%''')

O df composto pelos conectados possui shape (312990, 16)
O número de positivos para esse df é de 264623, representando 84.55%
------
O df composto pelos disconectados possui shape (305214, 16)
O número de positivos para esse df é de 3468, representando 1.14%


Fica clara a tendencia de resultados para cada um dos grupos.

In [ ]:
model = LogisticRegression(
            random_state=42,
            max_iter=500,
            C=0.1,
        )
num_folds = 5
cv_Precision = cv(p_connected.X_processed.values, p_connected.y_processed.values,  model, num_folds, metric=precision_score)
print(f'5-Fold Cross Validation Precision: {cv_Precision}')

model.fit(p_connected.X_processed.values, p_connected.y_processed.values.ravel())
y_pred = model.predict(p_connected.X_processed_test.values)
test_Precision= precision_score(p_connected.y_processed_test.values,y_pred)
print(f'Test Set Precision: {test_Precision:.4f}')

5-Fold Cross Validation Precision: [0.8433944157565233, 0.8475285084211248, 0.844847681513815, 0.842390310786106, 0.8454491455725122]
Test Set Precision: 0.8448


In [ ]:
f1_score(p_connected.y_processed_test.values,y_pred)

0.9155517017125515

In [ ]:
accuracy_score(p_connected.y_processed_test.values,y_pred)

0.8444224348631767

In [ ]:
{x:y for x,y in zip(p_connected.X_processed.columns,model.coef_[0])}

{'regional_1': -0.21660591397321086,
 'regional_2': -0.13142612507942772,
 'regional_3': 0.2380433018376329,
 'regional_4': 0.09436435819819264,
 'client_class_1': -0.282143122332071,
 'client_class_2': 0.10392942733676994,
 'client_class_3': 0.0998361296469653,
 'client_class_4': -0.130490019930821,
 'client_class_5': 0.14855014131166247,
 'client_class_7': 0.044693064950716545,
 'disconnection_executed_branch': -0.8887305906154995,
 'disconnection_executed_circuit_break': 0.08258586624609066,
 'disconnection_executed_pole': -0.05703567018397581,
 'disconnection_executed_remote': 0.8475560155365462,
 'total_debt': 0.19209319313222384,
 'total_bills': -6.722717903659356}

Iniciando a análise do df com registros que estão desconectados. Será necessário, dessa vez, um tratamento para o desbalanceamento da base

In [ ]:
model = LogisticRegression(
            random_state=42,
            max_iter=500,
            C=0.1,
        )
num_folds = 5
cv_Precision = cv(p_disconnected.X_processed.values, p_disconnected.y_processed.values,  model, num_folds, metric=precision_score)
print(f'5-Fold Cross Validation Precision: {cv_Precision}')

model.fit(p_disconnected.X_processed.values, p_disconnected.y_processed.values.ravel())
y_pred = model.predict(p_disconnected.X_processed_test.values)
test_Precision= precision_score(p_disconnected.y_processed_test.values,y_pred)
print(f'Test Set Precision: {test_Precision:.4f}')

c:\Users\leonardodomingos\.virtualenvs\LeonardoDomingosKearney-Q09UpS7u\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\leonardodomingos\.virtualenvs\LeonardoDomingosKearney-Q09UpS7u\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\leonardodomingos\.virtualenvs\LeonardoDomingosKearney-Q09UpS7u\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, m

5-Fold Cross Validation Precision: [0.0, 0.0, 0.0, 0.0, 0.0]
Test Set Precision: 0.0000


c:\Users\leonardodomingos\.virtualenvs\LeonardoDomingosKearney-Q09UpS7u\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Nova hipótese
Target = disconnection_executed

In [ ]:
df.columns

Index(['client_id', 'regional', 'site', 'client_class', 'client_subclass',
       'connection_type', 'connection_status', 'installment_value',
       'installment_status', 'total_debt', 'total_bills', 'disconnection_date',
       'disconnection_requested', 'contractor_id', 'disconnection_executed',
       'collected', 'total_debt_class'],
      dtype='object')

In [ ]:
categorical = [
    'regional', 
    'client_class',
    'connection_type',
    'total_debt_class'
]
numerical = [
    'total_bills'
]
datetime = [
    'disconnection_date'
]

pipeline_02 = Pipeline(df.loc[df.collected == 1].reset_index(drop=True), categorical, numerical, 'disconnection_executed')
pipeline_02.preprocessing(multiclass=True)

In [ ]:
model = LogisticRegression(
            random_state=42,
            max_iter=500,
            C=0.1,
        )
num_folds = 5
cv_Precision = cv(pipeline_02.X_processed.values, pipeline_02.y_processed,  model, num_folds, metric=accuracy_score)
print(f'5-Fold Cross Validation accuracy: {cv_Precision}')

model.fit(pipeline_02.X_processed.values, pipeline_02.y_processed)
y_pred = model.predict(pipeline_02.X_processed_test.values)
test_acc= accuracy_score(pipeline_02.y_processed_test,y_pred)
print(f'Test Set accuracy: {test_acc:.4f}')

5-Fold Cross Validation accuracy: [0.8644623254822551, 0.8673931578386443, 0.8651017798145583, 0.8682990514760738, 0.868352339337099]
Test Set accuracy: 0.8672


In [ ]:
for strategy in ['most_frequent', 'prior', 'stratified', 'uniform']:
    dummy_clf = DummyClassifier(strategy = strategy)
    dummy_clf.fit(pipeline_02.X_processed.values, pipeline_02.y_processed)
    y_pred_dummy = dummy_clf.predict(pipeline_02.y_processed_test)
    print(f'Utilizando a estratégia {strategy}: Acc = {accuracy_score(pipeline_02.y_processed_test,y_pred_dummy):.2f}')

Utilizando a estratégia most_frequent: Acc = 0.56
Utilizando a estratégia prior: Acc = 0.56
Utilizando a estratégia stratified: Acc = 0.47
Utilizando a estratégia uniform: Acc = 0.25


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(pipeline_02.y_processed_test, y_pred, target_names=pipeline_02.label_encoder.classes_))

               precision    recall  f1-score   support

       branch       0.00      0.00      0.00       377
circuit_break       0.77      0.95      0.85     20699
         pole       0.47      0.01      0.02      2262
       remote       0.96      0.88      0.92     30281

     accuracy                           0.87     53619
    macro avg       0.55      0.46      0.45     53619
 weighted avg       0.86      0.87      0.85     53619



c:\Users\leonardodomingos\.virtualenvs\LeonardoDomingosKearney-Q09UpS7u\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\leonardodomingos\.virtualenvs\LeonardoDomingosKearney-Q09UpS7u\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\leonardodomingos\.virtualenvs\LeonardoDomingosKearney-Q09UpS7u\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

A classificação funciona muito bem para remote e circuit_break, mas não para branch e pole que não possuem muita informação